# Importing the Modules

In [ ]:
import tensorflow as tf
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

# Data Preperation

In [ ]:
train_directory = 'data'

dataset = keras.preprocessing.image_dataset_from_directory(
    train_directory, 
    label_mode = None, 
    image_size=(64, 64), 
    batch_size=32, 
    shuffle=True

    ).map(lambda x: x/255.0)

Found 21551 files belonging to 1 classes.


# Modelling

In [ ]:
#Starting Noise Dimension
LATENT_DIM = 128

generator = keras.Sequential([
    layers.Input(shape=(LATENT_DIM)),
    #Taking the noise vector to a 8x8 image with 128 channel
    layers.Dense(8*8*128),
    layers.Reshape((8, 8, 128)),

    #Generator image need to larger so we use ConvTranspose istead of Conv
    layers.Conv2DTranspose(128, kernel_size = 4, strides = 2, padding = 'same'),
    layers.LeakyReLU(0.2),

    layers.Conv2DTranspose(256, kernel_size = 4, strides = 2, padding = 'same'),
    layers.LeakyReLU(0.2),

    layers.Conv2DTranspose(512, kernel_size = 4, strides = 2, padding = 'same'),
    layers.LeakyReLU(0.2),

    #This Conv layer doesn't multiply the image size
    layers.Conv2D(3, kernel_size = 5, padding='same', activation='sigmoid'),
])

generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8192)              1056768   
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 256)       0

# Discriminator

In [ ]:
discriminator = keras.Sequential([
    keras.Input(shape=(64, 64, 3)),
    layers.Conv2D(64, kernel_size = 4, strides = 2, padding = 'same'),
    layers.LeakyReLU(0.2),

    layers.Conv2D(128, kernel_size = 4, strides = 2, padding = 'same'),
    layers.LeakyReLU(0.2),

    layers.Conv2D(128, kernel_size = 4, strides = 2, padding = 'same'),
    layers.LeakyReLU(0.2),

    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(1, activation = 'sigmoid'),
    ])
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten (Flatten)           (None, 8192)             

# Training and Optimization

In [ ]:
loss_fn = keras.losses.BinaryCrossentropy(from_logits= True)

# Optimizer

In [ ]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 20

# Training Function

In [11]:
def train(dataset, epochs):
    print("Training Started...")
    for epoch in range(epochs):
        for idx, real in enumerate(tqdm(dataset)):
            batch_size = real.shape[0]
            random_latent_vectors = tf.random.normal(shape=(batch_size, LATENT_DIM))

            #Generating Fake Image
            with tf.GradientTape() as gen_tape:
                fake = generator(random_latent_vectors)

            with tf.GradientTape() as disc_tape:
                #Training Discriminator: max y*log(D(x)) + (1-y)*log(1 - D(G(z))), Here we are sending in y as 1 to get the log(D(x))
                loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
                loss_disc_fake = loss_fn(tf.zeros((batch_size, 1)), discriminator(fake))

                loss_disc = (loss_disc_fake + loss_disc_real) /2
            
            #Calculating the Gradient
            grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)

            #Updating the Discriminator
            opt_disc.apply_gradients(
                zip(grads, discriminator.trainable_weights)
                )

            ### Train the generator: min log(1-D(G(z))) it has saturation problem so we use log(D(G(z))) instead
            with tf.GradientTape() as gen_tape:
                fake = generator(random_latent_vectors)
                output = discriminator(fake)
                loss_gen = loss_fn(tf.ones(batch_size, 1), output)
            
            grads = gen_tape.gradient(loss_gen, generator.trainable_weights)

            opt_gen.apply_gradients(
                zip(grads, generator.trainable_weights)
                )
        # Display a generated image
        img = keras.preprocessing.image.array_to_img(generator(random_latent_vector)[0])
        img.save(f"generated_images/generated_img{epoch}_{idx}_.png")
        
        plt.imshow(img)
        plt.show()
        
        print("Epoch {}, D_loss: {}, G_loss: {}".format(epoch+1, loss_disc, loss_gen))
    return


In [12]:
train(dataset, EPOCHS)

Training Started...


 23%|██▎       | 152/674 [01:05<03:39,  2.37it/s]